In [2]:
#installing libraries
import sys
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install matplotlib --user
!{sys.executable} -m pip install scipy --user

  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d2/ab/43e678759326f728de861edbef34b8e2ad1b1490505f20e0d1f0716c3bf4/numpy-1.17.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/26/32b8464df2a97e6dd1b656ed26b2c194606c16fe163c695a992b36c11cdf/six-1.13.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/57/4f/dd381ecf6c6ab9bcdaa8ea912e866dedc6e696756156d8ecc087e20817e2/matplotlib-3.1.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached ht

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
import collections
import numbers

# Initial Conditions

In [4]:
S = 68.04
K = 67.50
σ = 0.2433
r = 0.0180
T= 173
δ = 0.0118

# Defining the Binomial Tree

In [19]:
import numpy as np

class Node:
    def __init__(self,left=None,right=None):
        self.S = 0
        self.δ = 0
        self.b = 0
        self.C = 0
        self.P = 0
        self.left = left
        self.right = right

class BinomialModel:
    def __init__(self,S,δ,r,σ,K,T):
        self.S = S
        self.δ = δ
        self.r = r
        self.σ = σ
        self.K = K
        self.T = T
        self.u = np.exp((r-δ)*T + σ*np.sqrt(T))
        self.d = 1/self.u
        self.p = (np.exp((r-δ)*T) - self.d)/(self.u-self.d)
        N = st.norm.cdf
        ϕ = st.norm.pdf
        d1 = (np.log(S/K)+(r-δ+0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        d2 = (np.log(S/K)+(r-δ-0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        self.C_Eur = S*np.exp(-δ*T)*N(d1)-K*np.exp(-r*T)*N(d2)
        self.P_Eur = K*np.exp(-r*T)*N(-d2)-S*np.exp(-δ*T)*N(-d1)
        self.Δ_Call=np.exp(-δ*T)*N(d1)
        self.Δ_Put=-np.exp(-δ*T)*N(-d1)
        self.Γ_Call=(1.0/(S*self.σ*np.sqrt(T)))*np.exp(-δ*T)*ϕ(d1)
        self.Γ_Put=self.Γ_Call
        #Scaled by 1/100
        self.ν_Call=(S*np.exp(-δ*T)*ϕ(d1)*np.sqrt(T))/100.0
        self.ν_Put=self.ν_Call
        #Scaled by 1/365
        self.Θ_Call=(δ*S*np.exp(-δ*T)*N(d1)-r*K*np.exp(-r*T)*N(d2)-(σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        self.Θ_Put=(r*K*np.exp(-r*T)*N(-d2)-δ*S*np.exp(-δ*T)*N(-d1)-(self.σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        #Scaled 1/10,000
        self.ρ_Call=(T*K*np.exp(-r*T)*N(d2))/10000.0
        self.ρ_Put=(-T*K*np.exp(-r*T)*N(-d2))/10000.0
        #Scaled 1/10,000
        self.Ψ_Call=(-T*S*np.exp(-δ*T)*N(d1))/10000.0
        self.Ψ_Put=(T*S*np.exp(-δ*T)*N(-d1))/10000.0
        
        

    #creates recombining tree
    def getTree(periods,americal=False):
        prevlevel = np.repeat(Node, periods)
        for i in range(0,periods): #fill out bottom level
            prevlevel[i].S = np.pow(self.u,i)*np.pow(self.d,(periods-i))*self.S
            prevlevel[i].C = max(0,prevlevel[i].S - self.K)

        for i in range(1,periods): #start at the bottom
            level = np.repeat(Node, periods - i)
            for j in range(1,periods-i):
                level[j].left = prevlevel[j]
                level[j].right = prevlevel[j+1]
                level[j].S = left.S/self.u
                level[j].C = (1-self.p)*prevlevel[j].C + self.p*prevlevel[j+1].C
                level[j].P = level[j].C - self.S*np.exp(-self.δ*self.T) + self.T*np.exp(-self.r*self.T)
                level[j].δ = (prevlevel[j+1].C - prevlevel[j].C)/(prevlevel[j+1].S - prevlevel[j].S)
                #TODO level[j].B
            prevlevel = level

        return prevlevel[0]




In [20]:
model = BinomialModel(S,δ,r,σ,K,T)